# C3RO demographics analysis base file generator

Uses publicly avaliable C3RO image files to generate CSV files which can be used for subsequent linear regression analysis.

Utilizes data from here: https://figshare.com/articles/dataset/Large-scale_crowdsourced_radiotherapy_segmentations_across_a_variety_of_cancer_sites/21074182.

Author: Kareem A. Wahid.

Last edited by Kareem Wahid on August, 23, 2022.

## Table of contents

1. [Helper Functions](#helper)

3. [Main code](#maincode)

In [1]:
# dependencies

import os, shutil
import SimpleITK as sitk # pip install SimpleITK
import numpy as np
import pandas as pd
from surface_distance import compute_dice_coefficient, compute_surface_distances, compute_robust_hausdorff, compute_surface_dice_at_tolerance, compute_average_surface_distance # installation instructions: https://github.com/deepmind/surface-distance
import itertools

## Helper Functions <a class="anchor" id="helper"></a>

Note: Most of this code is (unfortunatley) hard-coded to work with the C3RO data folder/file structures as downloaded from Figshare. If you make alterations to the way the folders are laid out it will probably break.

In [2]:
# adapted from Kiser et al. https://github.com/kkiser1/Autosegmentation-Spatial-Similarity-Metrics

def getEdgeOfMask(mask):
    '''
    Computes and returns edge of a segmentation mask
    '''
    # edge has the pixels which are at the edge of the mask
    edge = np.zeros_like(mask)
    
    # mask_pixels has the pixels which are inside the mask of the automated segmentation result
    mask_pixels = np.where(mask > 0)

    for idx in range(0,mask_pixels[0].size):

        x = mask_pixels[0][idx]
        y = mask_pixels[1][idx]
        z = mask_pixels[2][idx]

        # Count # pixels in 3x3 neighborhood that are in the mask
        # If sum < 27, then (x, y, z) is on the edge of the mask
        if mask[x-1:x+2, y-1:y+2, z-1:z+2].sum() < 27:
            edge[x,y,z] = 1
            
    return edge

def AddedPathLength(auto, gt):
    '''
    Returns the added path length, in pixels
    
    Steps:
    1. Find pixels at the edge of the mask for both auto and gt
    2. Count # pixels on the edge of gt that are not in the edge of auto
    '''
    
    # Check if auto and gt have same dimensions. If not, then raise a ValueError
    if auto.shape != gt.shape:
        raise ValueError('Shape of auto and gt must be identical!')

    # edge_auto has the pixels which are at the edge of the automated segmentation result
    edge_auto = getEdgeOfMask(auto)
    # edge_gt has the pixels which are at the edge of the ground truth segmentation
    edge_gt = getEdgeOfMask(gt)
    
    # Count # pixels on the edge of gt that are on not in the edge of auto
    apl = (edge_gt > edge_auto).astype(int).sum()
    
    return apl 

In [3]:
def pairwise_main(lst): # main function so you don't waste time 
    DSC_list = []
    HD95_list = []
    ASD_list = []
    APL_list = []
    ordered_list = itertools.combinations(lst,2)
    for i in ordered_list:
        file_1 = sitk.ReadImage(i[0])
        file_2 = sitk.ReadImage(i[1])
        
        spacing_mm = file_1.GetSpacing() # need spacing to calcualte distances
        
        mask_1 = sitk.GetArrayFromImage(file_1)
        mask_2 = sitk.GetArrayFromImage(file_2)
        
        DSC = compute_dice_coefficient(mask_1, mask_2)
        DSC_list.append(DSC)
        
        mask_1_bool = mask_1.astype(bool) # need to convert to bool format to work 
        mask_2_bool = mask_2.astype(bool)
        
        surface_distances = compute_surface_distances(mask_1_bool, mask_2_bool, spacing_mm) # need this for all calculations
        HD95 = compute_robust_hausdorff(surface_distances, 95)
        HD95_list.append(HD95)
        
        ASD_truth = compute_average_surface_distance(surface_distances)[0] #  A tuple with two float values: 1. the average distance (in mm) from the ground truth surface to thepredicted surface, 2. the average distance from the predicted surface to the ground truth surface.
        ASD_pred = compute_average_surface_distance(surface_distances)[1]
        ASD_mean = (ASD_truth + ASD_pred)/2
        ASD_list.append(ASD_mean)
        
        APL = AddedPathLength(mask_1, mask_2)
        APL_list.append(APL)
        
    return DSC_list, HD95_list, ASD_list, APL_list

def pairwise_SDSC(lst, tolerance): # do the same thing for SDSC
    SDSC_list = []
    ordered_list = itertools.combinations(lst,2)
    for i in ordered_list:
        file_1 = sitk.ReadImage(i[0])
        file_2 = sitk.ReadImage(i[1])
        
        spacing_mm = file_1.GetSpacing() # need spacing to calcualte distances
        
        mask_1 = sitk.GetArrayFromImage(file_1)
        mask_2 = sitk.GetArrayFromImage(file_2)
        
        mask_1_bool = mask_1.astype(bool) # need to convert to bool format to work 
        mask_2_bool = mask_2.astype(bool)
        
        surface_distances = compute_surface_distances(mask_1_bool, mask_2_bool, spacing_mm) # need this for all calculations
        
        SDSC = compute_surface_dice_at_tolerance(surface_distances, tolerance)
        
        SDSC_list.append(SDSC)
        
    return SDSC_list

In [4]:
def generate_expert_pairwise_df(site_path, ROI_list):
    
    expert_path = os.path.join(site_path, "Segmentations", "Expert")
    
    
    df = pd.DataFrame(columns=['ROI', 'DSC_list', 'HD95_list', 'ASD_list', 'APL_list'])
    counter = -1  

    for ROI in ROI_list:
        file_list = []

        ID_paths = [os.path.join(expert_path, folder, 'NIFTI') for folder in os.listdir(expert_path) if folder != "Consensus"]

        for ID_path in ID_paths:

            structures = os.listdir(ID_path)
            for structure in structures:
                structure_path = os.path.join(ID_path, structure)
                
                #new edit
                if ROI in structure.split('.')[0]:
                    file_list.append(structure_path)

        DSC_list, HD95_list, ASD_list, APL_list = pairwise_main(file_list)

        counter +=1
        df.loc[counter] = [ROI, DSC_list, HD95_list, ASD_list, APL_list]
        
    #SDSC
    SDSC_master_list = []
    for ROI in ROI_list:
        file_list = []

        ID_paths = [os.path.join(expert_path, folder, 'NIFTI') for folder in os.listdir(expert_path) if folder != "Consensus"]

        for ID_path in ID_paths:

            structures = os.listdir(ID_path)
            for structure in structures:
                structure_path = os.path.join(ID_path, structure)
                #new edit
                if ROI in structure.split('.')[0]:
                    file_list.append(structure_path)
                    
        tolerance = np.median(df[(df["ROI"] == ROI)]["ASD_list"].values[0]) # get tolerance
        SDSC_list = pairwise_SDSC(file_list, tolerance)
        SDSC_master_list.append(SDSC_list)
    
    df['SDSC_list'] = SDSC_master_list
        
    return df

In [5]:
def generate_nonexpert_vs_expertSTAPLE_df(site_path, ROI_list, expert_pairwise_df):

    nonexpert_path = os.path.join(site_path, "Segmentations", "Non-Expert") # this will be for all the non-experts

    expert_staple_path = os.path.join(site_path, "Segmentations", "Expert", "Consensus") # this will be just for STAPLE comparison structure 

    df = pd.DataFrame(columns=['Record ID', 'ROI', 'DSC', 'HD95', 'APL', 'SDSC'])
    counter = -1  

    ID_list = [folder for folder in os.listdir(nonexpert_path) if folder != "Consensus"]

    for ID in ID_list:
        
        for ROI in ROI_list:

            nonexpert_file_path = os.path.join(nonexpert_path, ID, 'NIFTI', ID+'_'+ROI+'.nii.gz')

            if os.path.isfile(nonexpert_file_path):

                file_1 = sitk.ReadImage(nonexpert_file_path)

                expert_file_path = os.path.join(expert_staple_path, ROI+ '_STAPLE.nii.gz')

                file_2 = sitk.ReadImage(expert_file_path) # this will for sure exist 

                mask_1 = sitk.GetArrayFromImage(file_1)
                mask_2 = sitk.GetArrayFromImage(file_2)

                DSC = compute_dice_coefficient(mask_1, mask_2)  

                # adding other metrics

                spacing_mm = file_1.GetSpacing() # need spacing to calcualte distances

                mask_1_bool = mask_1.astype(bool) # need to convert to bool format to work 
                mask_2_bool = mask_2.astype(bool)

                surface_distances = compute_surface_distances(mask_1_bool, mask_2_bool, spacing_mm) # need this for all calculations
                HD95 = compute_robust_hausdorff(surface_distances, 95)

                ASD_truth = compute_average_surface_distance(surface_distances)[0] #  A tuple with two float values: 1. the average distance (in mm) from the ground truth surface to thepredicted surface, 2. the average distance from the predicted surface to the ground truth surface.
                ASD_pred = compute_average_surface_distance(surface_distances)[1]
                ASD = (ASD_truth + ASD_pred)/2

                APL = AddedPathLength(mask_1, mask_2)

                tolerance = np.median(expert_pairwise_df[(expert_pairwise_df["ROI"] == ROI)]["ASD_list"].values[0]) # get tolerance
                SDSC = compute_surface_dice_at_tolerance(surface_distances, tolerance)

                counter +=1
                df.loc[counter] = [ID, ROI, DSC, HD95, APL, SDSC]
    
    return df


## Main code <a class="anchor" id="maincode"></a>

In [6]:
%%time

data_path = "Z:\\Kareem\\C3RO\\data_descriptor\\Organized_files_v4" # path to the data (top-level folder) from Figshare here

# disease site name mapped to list of corresponding ROIs and sheet name of excel file 
site_lists = [
    ['Breast', ['BrachialPlex_L','CTV_Ax','CTV_Chestwall','CTV_IMN','CTV_Sclav_LN','Heart', 'A_LAD'], 0],
    ['Sarcoma', ['GTV', 'CTV', 'Genitals'], 1],
    ['H&N', ['GTVp', 'GTVn', 'CTV1', 'CTV2', 'Brainstem', 'Glnd_Submand_L', 'Glnd_Submand_R', 'Larynx', 'Musc_Constrict', 'Parotid_L', 'Parotid_R'], 2],
    ['GYN', ['GTVn', 'CTVn_4500', 'CTVp_4500', 'Bowel_Small'], 3],
    ['GI', ['CTV_4500', 'CTV_5400', 'Bag_Bowel'], 4]
                ] 

binarized_metrics = [
    "DSC_binary",
    "SDSC_binary"
              ]

for site in site_lists:
    site_name = site[0]
    print(site_name) # debug
    ROI_list = site[1]
    sheet_number = site[2]
    
    site_path = os.path.join(data_path, site_name) # path to disease site folder
    
    df_expert_pairwise = generate_expert_pairwise_df(site_path, ROI_list) # calculate expert pairwise values for benchmarking
    
    df_nonexpert_vs_STAPLE = generate_nonexpert_vs_expertSTAPLE_df(site_path, ROI_list, df_expert_pairwise) # calculate individual non-experts vs expert consensus
    df_nonexpert_vs_STAPLE['Record ID'] = df_nonexpert_vs_STAPLE['Record ID'].astype('int64') # object to int
    
    ### integrate the cutoff values and categories into a 2 new columns
    for metric in binarized_metrics:
        metric_name = metric.split('_')[0]
        
        # for easier access to column names
        metric_median = metric_name+'_expert_IOV'
        metric_list = metric_name+'_list'
        
        df_expert_pairwise[metric_median] = df_expert_pairwise[metric_list].apply(np.median)
        
        # Merge the dataframes on the 'ROI' column
        df_nonexpert_vs_STAPLE = pd.merge(df_nonexpert_vs_STAPLE, df_expert_pairwise[['ROI', metric_median]], on='ROI', how='left')
        # Create a new column that's 1 if 'DSC' > 'IOV', and 0 otherwise
        df_nonexpert_vs_STAPLE[metric] = (df_nonexpert_vs_STAPLE[metric_name] > df_nonexpert_vs_STAPLE[metric_median]).astype(int)
        
    ### integrate the demographic variable data 

    # get excel file
    excel_file_path = os.path.join(data_path, "C3RO_RedCap_est7.30.22.xlsx") # path to excel file with observer demographic data
    excel_df = pd.read_excel(excel_file_path, sheet_name=sheet_number) # sheet names as defined above 
    excel_df = excel_df[excel_df["Category"] == "Non-expert"] # only pick non-experts
    
    # merge the excel into main dataframe
    df_final = pd.merge(df_nonexpert_vs_STAPLE, excel_df, on = "Record ID", how = 'left')

    # create total years of practice variable
    df_final['What year did you start practicing (graduate residency)?'] = pd.to_numeric(df_final['What year did you start practicing (graduate residency)?'], errors='coerce') # new, need to make sure it gives same value
    df_final['Total years of practice'] = 2022 - df_final['What year did you start practicing (graduate residency)?']

    # clean column names for ease of use later
    df_final.columns = df_final.columns.str.replace(' ', '_').str.replace('\W', '') 
    df_final = df_final.rename(columns={
        'Which_best_describes_your_primary_practice_': 'Practice_type', 
        'How_many_radiation_oncologist_colleagues_do_you_work_with_at_your_primary_site_excluding_you_': 'Colleague_num', 
        'WHITE_Which_categories_describe_you___Select_all_that_apply___choiceWhite': 'Race_white', 
        'Do_you_have_an_academic_affiliation': 'Academic_affiliation',
        "On_most_days_you_are_in_clinic_is_there_another_radiation_oncologist_on_site_with_you_": 'Colleague_presence'
    })
    df_final.columns = df_final.columns.str.replace("Which_disease_sites_do_you_treat___Select_all_that_apply_choice", "Treat_site_")
    
    ### write to CSV file
    csv_file_path = 'csv_files\\' + site_name + '.csv'
    df_final.to_csv(csv_file_path, index=False)
    
    

Breast


<timed exec>:60: FutureWarning: The default value of regex will change from True to False in a future version.


Sarcoma


<timed exec>:60: FutureWarning: The default value of regex will change from True to False in a future version.


H&N


<timed exec>:60: FutureWarning: The default value of regex will change from True to False in a future version.


GYN


<timed exec>:60: FutureWarning: The default value of regex will change from True to False in a future version.


GI
Wall time: 1h 10min 59s


<timed exec>:60: FutureWarning: The default value of regex will change from True to False in a future version.


In [7]:
df_final.head() # show a sample of what the dataframe looks like, if running in order this will be GI

,Record_ID,ROI,DSC,HD95,APL,SDSC,DSC_expert_IOV,DSC_binary,SDSC_expert_IOV,SDSC_binary,...,Treat_site_Genitourinary,Treat_site_Gynecologic,Treat_site_Head__Neck,Treat_site_Lung_Thoracic,Treat_site_Lymphoma_Leukemia,Treat_site_Metastatic,Treat_site_Pediatric,Treat_site_Sarcoma,Treat_site_Skin_Cutaneous,Total_years_of_practice
0,1051,CTV_4500,0.620026,31.249985,95424,0.571932,0.759432,0,0.704576,0,...,Unchecked,Unchecked,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,7.0
1,1051,CTV_5400,0.620685,20.255470,13187,0.940841,0.621064,0,0.782480,1,...,Unchecked,Unchecked,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,7.0
2,1051,Bag_Bowel,0.729188,16.248541,57411,0.707708,0.635979,1,0.646844,1,...,Unchecked,Unchecked,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,7.0
3,1073,CTV_4500,0.785915,23.457825,84038,0.752650,0.759432,1,0.704576,1,...,Checked,Checked,Checked,Checked,Checked,Checked,Unchecked,Unchecked,Unchecked,4.0
4,1073,CTV_5400,0.540104,22.821624,14670,0.911699,0.621064,0,0.782480,1,...,Checked,Checked,Checked,Checked,Checked,Checked,Unchecked,Unchecked,Unchecked,4.0
